In [ ]:
from flask import Flask, Response
from jetcam.csi_camera import CSICamera
import cv2

app = Flask(__name__)

# Initialize the camera
camera = CSICamera(width=224, height=224, capture_fps=30)

def gen_frames():  
    while True:
        # Capture frame-by-frame
        frame = camera.read()
        
        # Convert the frame to JPEG format
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        
        # Yield the output frame in byte format
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  

@app.route('/video_feed')
def video_feed():
    # Video streaming route
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.104:5000/ (Press CTRL+C to quit)
192.168.0.77 - - [23/Aug/2024 13:04:50] "GET /video_feed HTTP/1.1" 200 -


In [14]:
import requests
import time

def get_number(server_ip, number):
    start_time = time.time()  # Temps avant l'envoi de la requête
    response = requests.get(f'http://{server_ip}:5000/send_number/{number}')
    end_time = time.time()  # Temps après la réception de la réponse
    
    if response.status_code == 200:
        latency = end_time - start_time
        data = response.json()
        print(f"Received number: {data['number']}")
        print(f"Latency: {latency * 1000:.2f} ms")  # Affiche la latence en millisecondes
    else:
        print("Failed to retrieve number")

if __name__ == '__main__':
    server_ip = '192.168.0.77'  # Remplacez cela par l'IP du serveur
    for i in range (10):
        get_number(server_ip, 42)


Received number: 42
Latency: 33.22 ms
Received number: 42
Latency: 28.86 ms
Received number: 42
Latency: 25.86 ms
Received number: 42
Latency: 26.78 ms
Received number: 42
Latency: 23.13 ms
Received number: 42
Latency: 29.69 ms
Received number: 42
Latency: 24.93 ms
Received number: 42
Latency: 28.68 ms
Received number: 42
Latency: 30.93 ms
Received number: 42
Latency: 23.97 ms


In [2]:
import requests
import json

# Adresse du serveur Flask avec le schéma http://
url = "http://192.168.0.77:5000/receive_data"
data = {'x': 0.5}  # Test avec une valeur statique pour simplifier

try:
    response = requests.post(url, json=data)
    if response.status_code == 200:
        print("Data sent successfully")
    else:
        print(f"Failed to send data, status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Error sending data: {e}")


Data sent successfully
